# Ad-Hoc

## 1. Run Ad-Hoc

### 1.1. List servers in group

In [ ]:
ansible group_debian --list

### 1.2. Work with module

```bash
$ ansible <group1, group2, ..., groupn | all> -m <module> -a "command string"
```

For example:
```bash
$ ansible all -m shell -a "ps aux"
```

## 2. Modules

### 2.1. Ping

In [ ]:
ansible group_debian1 -m ping

### 1.2. Raw 

In [ ]:
ansible group_debian1 -m raw -a "ps aux"

### 1.3. Software repository

#### 1.3.1. Yum

#### 1.3.2. Apt 

### 1.4. Template

- Arguments
    - `src`: template file in current machine
    - `dest`: target file in remote server
    - `owner`: owner of target file
    - `mode`: mode of target file, canbe numbers (like `0644`) or letters (like `u=rw,g=r,o=r`), see also `chmod` command
    - `backup`: `ture` if create backup file if file content is changed

- Run template module

In [ ]:
ansible group_debian1 -m template \
    -a "src='./test_template.txt' dest='~/test_template.txt' owner='alvin' mode='u=rw,g=r,o=r' backup='yes'" \
    --extra-vars "dynamic_name=Alvin"

- Show generated files on remote server

In [ ]:
ansible group_debian1 -m raw -a "ls -al ~/test_template*"

- Show content of lastest generated file

In [ ]:
ansible group_debian1 -a "cat ~/test_template.txt"

- remove all generated files on remote server

In [ ]:
ansible group_debian1 -m raw -a "rm ~/test_template*"

### 1.5. Copy

- `src`: The source file on local machine, can be file or directory
- `content`: The content will be copied
- `dest`: The remote server to copied file
- `directory_mode`: The permission mode of remote directory
- `force`: Always copy file to target even file exist when value is 'yes'
- `others`: other options, see also `file` module

#### 1.5.1. Copy file

- Use copy module

In [ ]:
ansible group_debian1 -m copy -a "src='./ansible.cfg' dest='~/' force='yes'"

- Show copied file on remote server

In [ ]:
ansible group_debian1 -m raw -a "ls -al ~/ansible.*"

- Remove copied files

In [ ]:
ansible group_debian1 -m raw -a "rm ~/ansible.*"

#### 1.5.2. Copy files in folder

- Use copy module

In [ ]:
ansible group_debian1 -m copy -a "src='./hosts/' dest='~/target' force='yes'"

- Show copied folders

In [ ]:
ansible group_debian1 -m raw -a "ls -al ~/target"

- Remove copied files

In [ ]:
ansible group_debian1 -m raw -a "rm -rf ~/target*"

#### 1.5.3. Copy folder

- Use copy module

In [ ]:
ansible group_debian1 -m copy -a "src='./hosts' dest='~/target' force='yes'"

- Show copied folders

In [ ]:
ansible group_debian1 -m raw -a "ls -al ~/target"
ansible group_debian1 -m raw -a "ls -al ~/target/hosts"

- Remove copied files

In [ ]:
ansible group_debian1 -m raw -a "rm -rf ~/target*"

#### 1.5.4. Copy content

- Use copy module

In [ ]:
ansible group_debian1 -m copy -a "content='Hello World' dest='~/target.txt' force='yes'"

- Show copied folders

In [ ]:
ansible group_debian1 -m raw -a "cat ~/target.txt"

- Remove copied files

In [ ]:
ansible group_debian1 -m raw -a "rm target.txt"